# LUPI-SKD Full Training Notebook

## Overview
- **Goal**: Train Qwen3-4B with Korean cultural knowledge using LUPI-SKD
- **Teacher**: Qwen3-32B (sees Q+K)
- **Student**: Qwen3-4B-Thinking (sees Q only)
- **Data**: korean_culture_train_200.json (180 train / 20 val)

## Key Settings
- Top-K: 5 (fixed)
- Epochs: 3
- LR: 2e-6
- Grad Accum: 4

In [1]:
# Cell 1: Imports and Environment Setup
import sys
import os
import json
import random
import time
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass, field

# CUDA setup
os.environ["CUDA_VISIBLE_DEVICES"] = "5,6,7"

# ML imports
import torch
import torch.nn.functional as F
from torch.optim import AdamW
from transformers import AutoModelForCausalLM, AutoTokenizer, get_scheduler

# Logging
import wandb
from tqdm.auto import tqdm  # auto selects best backend

# Project root (relative to notebook location)
PROJECT_ROOT = Path.cwd().parent  # notebooks/ -> SKD_RAG_K/
sys.path.insert(0, str(PROJECT_ROOT))

# Import optimized rollout functions (v2)
from train.lupi_rollout_optimized import lupi_skd_rollout_optimized
# from train.lupi_rollout_compiled import lupi_skd_rollout_compiled

# Default to stable version (can switch to compiled for testing)
lupi_skd_rollout = lupi_skd_rollout_optimized
# lupi_skd_rollout = lupi_skd_rollout_compiled

# Keep original for A/B comparison
from train.lupi_rollout import lupi_skd_rollout_block as lupi_skd_rollout_original

# Seed for reproducibility
SEED = 42
torch.manual_seed(SEED)
random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

print(f"Project root: {PROJECT_ROOT}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA devices: {torch.cuda.device_count()}")

print(f"\n[Rollout Functions]")
print(f"  Default: lupi_skd_rollout_optimized (stable)")
print(f"  Available: lupi_skd_rollout_compiled (experimental)")
print(f"  Original: lupi_skd_rollout_original (for A/B testing)")

Project root: /home/eoeldroal/WorkPlace/SKD_RAG_K
PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA devices: 3

[Rollout Functions]
  Default: lupi_skd_rollout_optimized (stable)
  Available: lupi_skd_rollout_compiled (experimental)
  Original: lupi_skd_rollout_original (for A/B testing)


In [2]:
# Cell 2: Configuration Dataclass

@dataclass
class LUPISKDConfig:
    # Model paths
    teacher_model_name: str = "Qwen/Qwen3-32B"
    student_model_name: str = "Qwen/Qwen3-4B-Thinking-2507"
    
    # Data (use PROJECT_ROOT from Cell 1)
    data_path: str = str(PROJECT_ROOT / "data/korean_culture_train_200.json")
    train_val_split: float = 0.9  # 180 train, 20 val
    
    # SKD hyperparameters
    top_k: int = 15
    gamma: int = 5
    teacher_temperature: float = 0.7
    student_temperature: float = 0.7
    teacher_top_p: float = 0.95
    student_top_p: float = 0.95
    
    # Training
    epochs: int = 3
    learning_rate: float = 2e-6
    gradient_accumulation_steps: int = 4
    max_grad_norm: float = 1.0
    max_new_tokens: int = 8192
    warmup_ratio: float = 0.1
    
    # Checkpoint (use PROJECT_ROOT from Cell 1)
    checkpoint_dir: str = str(PROJECT_ROOT / "checkpoints/lupi_skd")
    save_every_epoch: bool = True
    
    # WandB
    wandb_project: str = "LUPI-SKD-Korean-Culture"
    wandb_run_name: str = None  # Auto-generated if None
    
    # Misc
    seed: int = 42
    verbose: bool = True
    log_every_steps: int = 5
    val_every_steps: int = 45  # Roughly once per epoch (180/4=45 steps)

config = LUPISKDConfig()

print("[Configuration]")
print(f"  Teacher: {config.teacher_model_name}")
print(f"  Student: {config.student_model_name}")
print(f"  Data path: {config.data_path}")
print(f"  Checkpoint dir: {config.checkpoint_dir}")
print(f"  Top-K: {config.top_k}")
print(f"  Epochs: {config.epochs}")
print(f"  LR: {config.learning_rate}")
print(f"  Grad Accum: {config.gradient_accumulation_steps}")

[Configuration]
  Teacher: Qwen/Qwen3-32B
  Student: Qwen/Qwen3-4B-Thinking-2507
  Data path: /home/eoeldroal/WorkPlace/SKD_RAG_K/data/korean_culture_train_200.json
  Checkpoint dir: /home/eoeldroal/WorkPlace/SKD_RAG_K/checkpoints/lupi_skd
  Top-K: 15
  Epochs: 3
  LR: 2e-06
  Grad Accum: 4


In [3]:
# Cell 3: Model Loading

print(f"Loading Teacher: {config.teacher_model_name}")
print(f"Loading Student: {config.student_model_name}")

tokenizer = AutoTokenizer.from_pretrained(config.teacher_model_name)
tokenizer.pad_token = tokenizer.eos_token

# Teacher: eval mode, no gradient
teacher_model = AutoModelForCausalLM.from_pretrained(
    config.teacher_model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="kernels-community/vllm-flash-attn3",
).eval()

for param in teacher_model.parameters():
    param.requires_grad = False

# Student: train mode
student_model = AutoModelForCausalLM.from_pretrained(
    config.student_model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="kernels-community/vllm-flash-attn3",
)

print(f"\n[Models Loaded]")
print(f"  Teacher params: {sum(p.numel() for p in teacher_model.parameters())/1e9:.2f}B")
print(f"  Student params: {sum(p.numel() for p in student_model.parameters())/1e9:.2f}B")
print(f"  Student trainable: {sum(p.numel() for p in student_model.parameters() if p.requires_grad)/1e9:.2f}B")

Loading Teacher: Qwen/Qwen3-32B
Loading Student: Qwen/Qwen3-4B-Thinking-2507


`torch_dtype` is deprecated! Use `dtype` instead!


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]


[Models Loaded]
  Teacher params: 32.76B
  Student params: 4.02B
  Student trainable: 4.02B


In [4]:
# Cell 4: Dataset Loading and Splitting

def load_and_split_data(config: LUPISKDConfig):
    """Load korean_culture_train_200.json and split 9:1."""
    with open(config.data_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Shuffle with fixed seed
    random.seed(config.seed)
    random.shuffle(data)
    
    split_idx = int(len(data) * config.train_val_split)
    train_data = data[:split_idx]  # 180 samples
    val_data = data[split_idx:]    # 20 samples
    
    print(f"[Data Loaded]")
    print(f"  Total samples: {len(data)}")
    print(f"  Train samples: {len(train_data)}")
    print(f"  Val samples: {len(val_data)}")
    
    # Category distribution
    train_cats = {}
    for sample in train_data:
        cat = sample.get('category', 'unknown')
        train_cats[cat] = train_cats.get(cat, 0) + 1
    print(f"\n  Train category distribution:")
    for cat, count in sorted(train_cats.items()):
        print(f"    {cat}: {count}")
    
    return train_data, val_data

train_data, val_data = load_and_split_data(config)

# Show sample
print(f"\n[Sample Data]")
sample = train_data[0]
print(f"  ID: {sample.get('id', 'N/A')}")
print(f"  Category: {sample.get('category', 'N/A')}")
print(f"  Query: {sample['query']}...")
print(f"  Knowledge: {sample['knowledge']}...")

[Data Loaded]
  Total samples: 200
  Train samples: 180
  Val samples: 20

  Train category distribution:
    역사/인물: 35
    음식/발효: 39
    전통예절: 32
    지리/지역: 38
    현대문화: 36

[Sample Data]
  ID: history_027
  Category: 역사/인물
  Query: 1895년 을미사변 이후 을미의병이 일어난 경위와 주요 의병장들의 활동은 어떠했나요?...
  Knowledge: 을미의병은 1895년(을미년) 을미사변과 단발령에 항거하여 일어난 의병 운동이다. 을미사변은 1895년 10월 8일(음력 8월 20일) 일본 공사 미우라 고로의 지휘 아래 일본군이 경복궁에 난입하여 명성황후(민비)를 시해한 사건이다. 이 소식이 알려지면서 국민적 분노가 폭발하였고, 김홍집 친일 내각이 11월 단발령을 시행하자 유생들이 의병을 일으켰다. 충청남도에서 문석봉이 9월 18일 최초로 의병을 봉기하였고, 11월 충청도 제천에서는 유인석이 '거의토적(擧義討賊)'의 기치를 올리며 의병을 조직하였다. 강원도 강릉에서는 민용호가 1896년 1월 30일 관동9군도창의소를 설치하고 강릉부 경무관 고준석을 처단하였다. 의병들은 지방 도시를 공략하여 친일 관리와 일본인을 처단하였으며, 의병 수는 최고 5만 명에 달하였다. 그러나 1896년 2월 고종이 러시아 공사관으로 피신하는 아관파천 이후, 고종이 의병 해산을 권고하는 조칙을 내리자 의병들은 자진 해산하였다. 을미의병은 민족의 자주성을 지키려는 자발적 무장 항쟁이었으며, 이후 을사의병, 정미의병으로 이어졌다....


In [5]:
# Cell 5: LUPI Input Builder (from test_lupi_skd.ipynb)

def build_lupi_inputs(query: str, knowledge: str, tokenizer):
    """Build LUPI inputs: Teacher sees Q+K, Student sees Q only."""
    teacher_messages = [
        {"role": "system", "content": knowledge},
        {"role": "user", "content": query},
    ]
    student_messages = [
        {"role": "user", "content": query},
    ]

    teacher_inputs = tokenizer.apply_chat_template(
        teacher_messages,
        return_dict=True,
        tokenize=True,
        return_tensors="pt",
        add_generation_prompt=True,
        enable_thinking=True,
    )
    student_inputs = tokenizer.apply_chat_template(
        student_messages,
        return_dict=True,
        tokenize=True,
        return_tensors="pt",
        add_generation_prompt=True,
        enable_thinking=True,
    )
    return teacher_inputs, student_inputs

# Test
test_t, test_s = build_lupi_inputs(train_data[0]['query'], train_data[0]['knowledge'], tokenizer)
print(f"[LUPI Input Test]")
print(f"  Teacher input tokens: {test_t['input_ids'].shape[1]}")
print(f"  Student input tokens: {test_s['input_ids'].shape[1]}")
print(f"  Knowledge tokens (diff): {test_t['input_ids'].shape[1] - test_s['input_ids'].shape[1]}")

[LUPI Input Test]
  Teacher input tokens: 468
  Student input tokens: 47
  Knowledge tokens (diff): 421


In [ ]:
# Cell 5.5: 🔍 OPTIMIZED ROLLOUT DEBUG & A/B COMPARISON
# 목적: 1) 최적화된 rollout 테스트, 2) 기존 버전과 속도 비교, 3) 파이프라인 검증

print("="*70)
print("🔍 OPTIMIZED LUPI-SKD ROLLOUT DEBUG")
print("="*70)

# ============================================================
# 1. 샘플 선택 및 프롬프트 구분 검증
# ============================================================
debug_sample = train_data[0]
print(f"\n[1] 샘플 정보")
print(f"    ID: {debug_sample.get('id', 'N/A')}")
print(f"    Category: {debug_sample.get('category', 'N/A')}")
print(f"    Query 길이: {len(debug_sample['query'])} chars")
print(f"    Knowledge 길이: {len(debug_sample['knowledge'])} chars")

# LUPI 입력 생성
teacher_inputs, student_inputs = build_lupi_inputs(
    debug_sample['query'], debug_sample['knowledge'], tokenizer
)

print(f"\n[2] 토큰화된 입력 비교")
print(f"    Teacher input tokens: {teacher_inputs['input_ids'].shape[1]}")
print(f"    Student input tokens: {student_inputs['input_ids'].shape[1]}")
print(f"    차이 (Knowledge tokens): {teacher_inputs['input_ids'].shape[1] - student_inputs['input_ids'].shape[1]}")

# ============================================================
# 2. A/B 속도 비교 테스트
# ============================================================
print(f"\n{'='*70}")
print("[3] 🏎️ A/B SPEED COMPARISON (Original vs Optimized)")
print("="*70)

DEBUG_MAX_TOKENS = 1024  # 속도 비교용

# --- A) Original Version ---
print(f"\n  [A] ORIGINAL (lupi_skd_rollout_original)")
print(f"      max_new_tokens: {DEBUG_MAX_TOKENS}")
torch.cuda.synchronize()
start_original = time.time()

with torch.no_grad():
    _, _, tokens_original, text_original = lupi_skd_rollout_original(
        teacher_model=teacher_model,
        student_model=student_model,
        teacher_input_ids=teacher_inputs['input_ids'],
        student_input_ids=student_inputs['input_ids'],
        max_new_tokens=DEBUG_MAX_TOKENS,
        top_k=config.top_k,
        gamma=config.gamma,
        teacher_temperature=config.teacher_temperature,
        student_temperature=config.student_temperature,
        teacher_top_p=config.teacher_top_p,
        student_top_p=config.student_top_p,
        tokenizer=tokenizer,
        verbose=False,
    )

torch.cuda.synchronize()
time_original = time.time() - start_original
tokens_per_sec_original = len(tokens_original) / time_original

print(f"      Time: {time_original:.2f}s")
print(f"      Tokens: {len(tokens_original)}")
print(f"      Speed: {tokens_per_sec_original:.1f} tok/s")

# --- B) Optimized Version ---
print(f"\n  [B] OPTIMIZED (lupi_skd_rollout_optimized)")
print(f"      max_new_tokens: {DEBUG_MAX_TOKENS}")
torch.cuda.synchronize()
start_optimized = time.time()

with torch.no_grad():
    _, _, tokens_optimized, text_optimized, debug_stats = lupi_skd_rollout_optimized(
        teacher_model=teacher_model,
        student_model=student_model,
        teacher_input_ids=teacher_inputs['input_ids'],
        student_input_ids=student_inputs['input_ids'],
        max_new_tokens=DEBUG_MAX_TOKENS,
        top_k=config.top_k,
        gamma=config.gamma,
        teacher_temperature=config.teacher_temperature,
        student_temperature=config.student_temperature,
        teacher_top_p=config.teacher_top_p,
        student_top_p=config.student_top_p,
        tokenizer=tokenizer,
        verbose=False,
    )

torch.cuda.synchronize()
time_optimized = time.time() - start_optimized
tokens_per_sec_optimized = len(tokens_optimized) / time_optimized

print(f"      Time: {time_optimized:.2f}s")
print(f"      Tokens: {len(tokens_optimized)}")
print(f"      Speed: {tokens_per_sec_optimized:.1f} tok/s")

# --- Comparison Summary ---
speedup = time_original / time_optimized if time_optimized > 0 else 0
print(f"\n  [COMPARISON SUMMARY]")
print(f"      Speedup: {speedup:.2f}x {'✅' if speedup > 1.5 else '⚠️'}")
print(f"      Original: {tokens_per_sec_original:.1f} tok/s")
print(f"      Optimized: {tokens_per_sec_optimized:.1f} tok/s")
print(f"      Time saved: {time_original - time_optimized:.2f}s ({(1 - time_optimized/time_original)*100:.1f}%)")

# ============================================================
# 3. 최적화 버전 상세 통계
# ============================================================
print(f"\n{'='*70}")
print("[4] 📊 OPTIMIZED VERSION DETAILED STATS")
print("="*70)

print(f"  Total blocks: {debug_stats['total_blocks']}")
print(f"  Total drafted: {debug_stats['total_drafted']}")
print(f"  Total accepted: {debug_stats['total_accepted']}")
print(f"  Total rejected: {debug_stats['total_rejected']}")
print(f"  Avg acceptance rate: {debug_stats['avg_acceptance_rate']:.1%}")
print(f"  Teacher forward calls: {debug_stats['teacher_forward_calls']}")
print(f"  Student forward calls: {debug_stats['student_forward_calls']}")

# 기존 버전이라면 Teacher forward = gamma * blocks 번 호출되었을 것
estimated_original_teacher_calls = debug_stats['total_blocks'] * config.gamma
print(f"\n  [Forward Call Comparison]")
print(f"    Original (estimated): ~{estimated_original_teacher_calls} Teacher calls")
print(f"    Optimized (actual): {debug_stats['teacher_forward_calls']} Teacher calls")
print(f"    Reduction: {(1 - debug_stats['teacher_forward_calls']/estimated_original_teacher_calls)*100:.1f}%")

# Per-block acceptance rates
if debug_stats['block_stats']:
    print(f"\n  [Per-Block Acceptance Rates (first 10)]")
    for i, bs in enumerate(debug_stats['block_stats'][:10]):
        bar = '█' * int(bs['acceptance_rate'] * 10) + '░' * (10 - int(bs['acceptance_rate'] * 10))
        print(f"    Block {i+1:2d}: {bar} {bs['n_matches']}/{bs['gamma']} ({bs['acceptance_rate']:.0%})")

# ============================================================
# 4. 생성된 텍스트 비교
# ============================================================
print(f"\n{'='*70}")
print("[5] 📄 GENERATED TEXT COMPARISON")
print("="*70)

print(f"\n  [Original ({len(tokens_original)} tokens)]")
print(f"  {text_original}")

print(f"\n  [Optimized ({len(tokens_optimized)} tokens)]")
print(f"  {text_optimized}")

# Check if outputs are similar (they won't be identical due to sampling)
print(f"\n  Note: Outputs differ due to stochastic sampling, but quality should be comparable.")

# ============================================================
# 5. 검증 완료
# ============================================================
print(f"\n{'='*70}")
print("✅ DEBUG COMPLETE - 최적화 검증 완료!")
print("="*70)
print(f"\n요약:")
print(f"  - 속도 향상: {speedup:.2f}x {'✅ 성공' if speedup > 1.5 else '⚠️ 추가 확인 필요'}")
print(f"  - Teacher forward 감소: {(1 - debug_stats['teacher_forward_calls']/estimated_original_teacher_calls)*100:.1f}%")
print(f"  - Acceptance Rate: {debug_stats['avg_acceptance_rate']:.1%}")
print(f"  - 파이프라인 동작: ✅ ({len(tokens_optimized)} tokens 생성)")

# 글로벌 rollout 함수를 최적화 버전으로 확정
if speedup > 1.0:
    print(f"\n  ✅ Using optimized version (lupi_skd_rollout_optimized) for training.")

🔍 OPTIMIZED LUPI-SKD ROLLOUT DEBUG

[1] 샘플 정보
    ID: history_027
    Category: 역사/인물
    Query 길이: 47 chars
    Knowledge 길이: 541 chars

[2] 토큰화된 입력 비교
    Teacher input tokens: 468
    Student input tokens: 47
    차이 (Knowledge tokens): 421

[3] 🏎️ A/B SPEED COMPARISON (Original vs Optimized)

  [A] ORIGINAL (lupi_skd_rollout_original)
      max_new_tokens: 1024


In [7]:
# Cell 6: Loss Computation Functions (extended from test_lupi_skd.ipynb)

def _extract_generation_logits(logits, prompt_len):
    """Extract logits for generation positions (aligned with tokens)."""
    if prompt_len < 1:
        raise ValueError('prompt_len must be >= 1')
    return logits[:, prompt_len - 1:-1, :]

def compute_lupi_kl_loss(
    teacher_model,
    student_model,
    teacher_full_ids,
    student_full_ids,
    teacher_inputs,
    student_inputs,
    reduction='batchmean',
):
    """Compute KL(Student || Teacher) with gradients flowing to student only.
    
    Returns dict with:
        - loss: KL divergence (scalar)
        - per_token_kl: list of per-token KL values
        - gen_len: generation length used
        - teacher_entropy: average teacher distribution entropy
        - student_entropy: average student distribution entropy
    """
    device = next(student_model.parameters()).device
    
    with torch.no_grad():
        teacher_outputs = teacher_model(
            input_ids=teacher_full_ids.to(device),
            attention_mask=torch.ones_like(teacher_full_ids).to(device),
        )

    student_outputs = student_model(
        input_ids=student_full_ids.to(device),
        attention_mask=torch.ones_like(student_full_ids).to(device),
    )

    teacher_logits = _extract_generation_logits(
        teacher_outputs.logits, teacher_inputs['input_ids'].shape[1]
    )
    student_logits = _extract_generation_logits(
        student_outputs.logits, student_inputs['input_ids'].shape[1]
    )

    gen_len = min(teacher_logits.shape[1], student_logits.shape[1])
    teacher_logps = teacher_logits[:, :gen_len, :].log_softmax(dim=-1).detach()
    student_logps = student_logits[:, :gen_len, :].log_softmax(dim=-1)

    kl_loss = F.kl_div(student_logps, teacher_logps, reduction=reduction, log_target=True)

    # Per-token KL for analysis
    with torch.no_grad():
        per_token_kl = (
            F.kl_div(student_logps, teacher_logps, reduction='none', log_target=True)
            .sum(dim=-1)[0]
            .cpu()
            .tolist()
        )
        
        # Entropy metrics
        teacher_probs = teacher_logits[:, :gen_len, :].softmax(dim=-1)
        teacher_entropy = -(teacher_probs * teacher_probs.clamp(min=1e-10).log()).sum(dim=-1).mean().item()
        
        student_probs = student_logits[:, :gen_len, :].softmax(dim=-1)
        student_entropy = -(student_probs * student_probs.clamp(min=1e-10).log()).sum(dim=-1).mean().item()
    
    return {
        'loss': kl_loss,
        'per_token_kl': per_token_kl,
        'gen_len': gen_len,
        'teacher_entropy': teacher_entropy,
        'student_entropy': student_entropy,
    }

print("[Loss function defined]")

[Loss function defined]


In [8]:
# Cell 7: Validation Function (with detailed output) - OPTIMIZED VERSION

def validate_with_details(
    teacher_model,
    student_model,
    val_data,
    config,
    tokenizer,
    num_generate_samples=3,
):
    """Run validation with detailed logging using OPTIMIZED rollout.
    
    Uses lupi_skd_rollout_optimized for faster validation.
    """
    student_model.eval()
    val_losses = []
    val_gen_lens = []
    teacher_entropies = []
    student_entropies = []
    per_sample_results = []
    
    # Rollout statistics
    total_acceptance_rate = 0.0
    total_teacher_forward_calls = 0
    
    samples_for_logging = []
    
    print(f"  Running validation on {len(val_data)} samples (using optimized rollout)...")
    
    with torch.no_grad():
        for i, sample in enumerate(tqdm(val_data, desc="Validation", leave=False)):
            sample_start = time.time()
            
            teacher_inputs, student_inputs = build_lupi_inputs(
                sample['query'], sample['knowledge'], tokenizer
            )
            
            # Optimized Rollout (returns debug_stats)
            teacher_full_ids, student_full_ids, gen_tokens, gen_str, rollout_stats = lupi_skd_rollout(
                teacher_model=teacher_model,
                student_model=student_model,
                teacher_input_ids=teacher_inputs['input_ids'],
                student_input_ids=student_inputs['input_ids'],
                max_new_tokens=config.max_new_tokens,
                top_k=config.top_k,
                gamma=config.gamma,
                teacher_temperature=config.teacher_temperature,
                student_temperature=config.student_temperature,
                teacher_top_p=config.teacher_top_p,
                student_top_p=config.student_top_p,
                tokenizer=tokenizer,
                verbose=False,
            )
            
            # Loss (no grad for validation)
            loss_dict = compute_lupi_kl_loss(
                teacher_model, student_model,
                teacher_full_ids, student_full_ids,
                teacher_inputs, student_inputs,
            )
            
            sample_time = time.time() - sample_start
            
            val_losses.append(loss_dict['loss'].item())
            val_gen_lens.append(loss_dict['gen_len'])
            teacher_entropies.append(loss_dict['teacher_entropy'])
            student_entropies.append(loss_dict['student_entropy'])
            
            # Accumulate rollout stats
            total_acceptance_rate += rollout_stats.get('avg_acceptance_rate', 0)
            total_teacher_forward_calls += rollout_stats.get('teacher_forward_calls', 0)
            
            # Per-sample result
            per_sample_results.append({
                'id': sample.get('id', f'val_{i}'),
                'category': sample.get('category', 'unknown'),
                'loss': loss_dict['loss'].item(),
                'gen_len': loss_dict['gen_len'],
                'teacher_entropy': loss_dict['teacher_entropy'],
                'student_entropy': loss_dict['student_entropy'],
                'acceptance_rate': rollout_stats.get('avg_acceptance_rate', 0),
            })
            
            # Print per-sample info
            print(f"    Val[{i+1}/{len(val_data)}] {sample.get('id', 'N/A'):20s} | "
                  f"Loss: {loss_dict['loss'].item():.4f} | "
                  f"GenLen: {loss_dict['gen_len']:4d} | "
                  f"AccRate: {rollout_stats.get('avg_acceptance_rate', 0):.1%} | "
                  f"Time: {sample_time:.1f}s")
            
            # Collect sample for logging
            if i < num_generate_samples:
                samples_for_logging.append({
                    'id': sample.get('id', f'val_{i}'),
                    'category': sample.get('category', 'unknown'),
                    'query': sample['query'],
                    'knowledge': sample['knowledge'][:200] + '...' if len(sample['knowledge']) > 200 else sample['knowledge'],
                    'generated': gen_str[:500] if len(gen_str) > 500 else gen_str,
                    'gen_len': len(gen_tokens),
                    'loss': loss_dict['loss'].item(),
                    'acceptance_rate': rollout_stats.get('avg_acceptance_rate', 0),
                })
    
    student_model.train()
    
    # Compute statistics
    avg_loss = sum(val_losses) / len(val_losses)
    std_loss = (sum((l - avg_loss)**2 for l in val_losses) / len(val_losses)) ** 0.5
    avg_acceptance_rate = total_acceptance_rate / len(val_data)
    
    # Category breakdown
    category_stats = {}
    for r in per_sample_results:
        cat = r['category']
        if cat not in category_stats:
            category_stats[cat] = {'losses': [], 'gen_lens': [], 'acceptance_rates': []}
        category_stats[cat]['losses'].append(r['loss'])
        category_stats[cat]['gen_lens'].append(r['gen_len'])
        category_stats[cat]['acceptance_rates'].append(r['acceptance_rate'])
    
    print(f"\n  [Validation Statistics]")
    print(f"    Overall: Loss = {avg_loss:.4f} ± {std_loss:.4f}")
    print(f"    Avg Acceptance Rate: {avg_acceptance_rate:.1%}")
    print(f"    Total Teacher Forward Calls: {total_teacher_forward_calls}")
    print(f"    Per-category breakdown:")
    for cat, stats in sorted(category_stats.items()):
        cat_avg = sum(stats['losses']) / len(stats['losses'])
        cat_gen = sum(stats['gen_lens']) / len(stats['gen_lens'])
        cat_acc = sum(stats['acceptance_rates']) / len(stats['acceptance_rates'])
        print(f"      {cat:15s}: Loss = {cat_avg:.4f}, Avg GenLen = {cat_gen:.1f}, AccRate = {cat_acc:.1%} (n={len(stats['losses'])})")
    
    return {
        'val_loss': avg_loss,
        'val_loss_std': std_loss,
        'val_gen_len_avg': sum(val_gen_lens) / len(val_gen_lens),
        'val_teacher_entropy': sum(teacher_entropies) / len(teacher_entropies),
        'val_student_entropy': sum(student_entropies) / len(student_entropies),
        'val_acceptance_rate': avg_acceptance_rate,
        'samples': samples_for_logging,
        'per_sample_results': per_sample_results,
        'category_stats': category_stats,
    }

print("[Validation function defined with OPTIMIZED rollout]")

[Validation function defined with OPTIMIZED rollout]


In [9]:
# Cell 8: WandB Initialization

def init_wandb(config: LUPISKDConfig):
    """Initialize WandB with comprehensive config."""
    run_name = config.wandb_run_name or f"lupi-skd-k{config.top_k}-lr{config.learning_rate}-{datetime.now().strftime('%Y%m%d_%H%M')}"
    
    wandb.init(
        project=config.wandb_project,
        name=run_name,
        config={
            # Model
            "teacher_model": config.teacher_model_name,
            "student_model": config.student_model_name,
            
            # SKD
            "top_k": config.top_k,
            "gamma": config.gamma,
            "teacher_temperature": config.teacher_temperature,
            "student_temperature": config.student_temperature,
            "teacher_top_p": config.teacher_top_p,
            "student_top_p": config.student_top_p,
            
            # Training
            "epochs": config.epochs,
            "learning_rate": config.learning_rate,
            "gradient_accumulation_steps": config.gradient_accumulation_steps,
            "max_grad_norm": config.max_grad_norm,
            "max_new_tokens": config.max_new_tokens,
            "warmup_ratio": config.warmup_ratio,
            
            # Data
            "train_samples": int(200 * config.train_val_split),
            "val_samples": int(200 * (1 - config.train_val_split)),
            "data_source": "korean_culture_train_200.json",
            
            # Misc
            "seed": config.seed,
        },
        tags=["LUPI-SKD", "Korean-Culture", "Qwen3"],
    )
    
    return run_name

run_name = init_wandb(config)
print(f"[WandB Initialized]")
print(f"  Project: {config.wandb_project}")
print(f"  Run: {run_name}")

wandb: Currently logged in as: eoeldroal (eoeldroal-sogang-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[WandB Initialized]
  Project: LUPI-SKD-Korean-Culture
  Run: lupi-skd-k5-lr2e-06-20251205_1737


In [10]:
# Cell 9: Optimizer and Scheduler Setup

def setup_training(student_model, config, num_training_steps):
    """Setup optimizer and scheduler."""
    optimizer = AdamW(
        student_model.parameters(),
        lr=config.learning_rate,
        weight_decay=0.01,
    )
    
    num_warmup_steps = int(num_training_steps * config.warmup_ratio)
    
    scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=num_training_steps,
    )
    
    return optimizer, scheduler

# Calculate total steps
steps_per_epoch = len(train_data) // config.gradient_accumulation_steps
total_steps = steps_per_epoch * config.epochs

print(f"[Training Setup]")
print(f"  Train samples: {len(train_data)}")
print(f"  Grad accum: {config.gradient_accumulation_steps}")
print(f"  Steps per epoch: {steps_per_epoch}")
print(f"  Total training steps: {total_steps}")
print(f"  Warmup steps: {int(total_steps * config.warmup_ratio)}")

optimizer, scheduler = setup_training(student_model, config, total_steps)

[Training Setup]
  Train samples: 180
  Grad accum: 4
  Steps per epoch: 45
  Total training steps: 135
  Warmup steps: 13


In [11]:
# Cell 10: Checkpoint Functions

def save_checkpoint(student_model, tokenizer, config, epoch, step, val_loss):
    """Save checkpoint with metadata."""
    ckpt_dir = Path(config.checkpoint_dir)
    ckpt_dir.mkdir(parents=True, exist_ok=True)
    
    save_path = ckpt_dir / f"epoch{epoch}_step{step}_valloss{val_loss:.4f}"
    save_path.mkdir(exist_ok=True)
    
    student_model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)
    
    # Save metadata
    metadata = {
        "epoch": epoch,
        "step": step,
        "val_loss": val_loss,
        "timestamp": datetime.now().isoformat(),
        "config": {
            "top_k": config.top_k,
            "gamma": config.gamma,
            "learning_rate": config.learning_rate,
            "teacher_model": config.teacher_model_name,
            "student_model": config.student_model_name,
        }
    }
    with open(save_path / "metadata.json", 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f"  [Checkpoint saved: {save_path.name}]")
    return str(save_path)

print(f"[Checkpoint directory: {config.checkpoint_dir}]")

[Checkpoint directory: /home/eoeldroal/WorkPlace/SKD_RAG_K/checkpoints/lupi_skd]


In [12]:
# Cell 11: Main Training Loop (with OPTIMIZED rollout)

def train_lupi_skd(
    teacher_model,
    student_model,
    train_data,
    val_data,
    config,
    tokenizer,
    optimizer,
    scheduler,
):
    """Main LUPI-SKD training loop with OPTIMIZED rollout and detailed logging."""
    student_model.train()
    
    global_step = 0
    accumulated_loss = 0.0
    accumulated_samples = 0
    
    # Metrics accumulators for logging
    step_gen_lens = []
    step_teacher_entropies = []
    step_student_entropies = []
    step_acceptance_rates = []
    step_teacher_forward_calls = []
    
    best_val_loss = float('inf')
    training_log = []
    
    # Initial validation before training
    print("\n" + "="*70)
    print("[INITIAL VALIDATION] Before training starts")
    print("="*70)
    val_results = validate_with_details(
        teacher_model, student_model, val_data,
        config, tokenizer, num_generate_samples=2
    )
    print(f"  Initial Val Loss: {val_results['val_loss']:.4f}")
    print(f"  Initial Val Gen Len: {val_results['val_gen_len_avg']:.1f}")
    print(f"  Initial Val Acceptance Rate: {val_results.get('val_acceptance_rate', 0):.1%}")
    print("="*70 + "\n")
    
    for epoch in range(1, config.epochs + 1):
        print(f"\n{'#'*70}")
        print(f"#  EPOCH {epoch}/{config.epochs}")
        print(f"{'#'*70}")
        
        # Shuffle training data each epoch
        random.shuffle(train_data)
        
        epoch_start_time = time.time()
        epoch_losses = []
        epoch_gen_lens = []
        epoch_acceptance_rates = []
        
        pbar = tqdm(enumerate(train_data), total=len(train_data), desc=f"Epoch {epoch}")
        
        for sample_idx, sample in pbar:
            sample_start_time = time.time()
            
            # Build LUPI inputs
            teacher_inputs, student_inputs = build_lupi_inputs(
                sample['query'], sample['knowledge'], tokenizer
            )
            
            # LUPI-SKD Rollout using OPTIMIZED version (no grad for rollout itself)
            with torch.no_grad():
                teacher_full_ids, student_full_ids, gen_tokens, gen_str, rollout_stats = lupi_skd_rollout(
                    teacher_model=teacher_model,
                    student_model=student_model,
                    teacher_input_ids=teacher_inputs['input_ids'],
                    student_input_ids=student_inputs['input_ids'],
                    max_new_tokens=config.max_new_tokens,
                    top_k=config.top_k,
                    gamma=config.gamma,
                    teacher_temperature=config.teacher_temperature,
                    student_temperature=config.student_temperature,
                    teacher_top_p=config.teacher_top_p,
                    student_top_p=config.student_top_p,
                    tokenizer=tokenizer,
                    verbose=False,
                )
            
            # Compute KL Loss (with grad)
            loss_dict = compute_lupi_kl_loss(
                teacher_model, student_model,
                teacher_full_ids, student_full_ids,
                teacher_inputs, student_inputs,
            )
            
            loss = loss_dict['loss'] / config.gradient_accumulation_steps
            loss.backward()
            
            sample_time = time.time() - sample_start_time
            
            # Accumulate metrics
            accumulated_loss += loss_dict['loss'].item()
            accumulated_samples += 1
            epoch_losses.append(loss_dict['loss'].item())
            epoch_gen_lens.append(loss_dict['gen_len'])
            epoch_acceptance_rates.append(rollout_stats.get('avg_acceptance_rate', 0))
            
            step_gen_lens.append(loss_dict['gen_len'])
            step_teacher_entropies.append(loss_dict['teacher_entropy'])
            step_student_entropies.append(loss_dict['student_entropy'])
            step_acceptance_rates.append(rollout_stats.get('avg_acceptance_rate', 0))
            step_teacher_forward_calls.append(rollout_stats.get('teacher_forward_calls', 0))
            
            # Update progress bar
            pbar.set_postfix({
                'loss': f"{loss_dict['loss'].item():.4f}",
                'gen_len': loss_dict['gen_len'],
                'acc_rate': f"{rollout_stats.get('avg_acceptance_rate', 0):.0%}",
                'time': f"{sample_time:.1f}s"
            })
            
            # Detailed sample logging (every N samples)
            if (sample_idx + 1) % 10 == 0 or sample_idx == 0:
                print(f"\n  [Sample {sample_idx+1}/{len(train_data)}] ID: {sample.get('id', 'N/A')}")
                print(f"    Category: {sample.get('category', 'N/A')}")
                print(f"    Query: {sample['query'][:60]}...")
                print(f"    Loss: {loss_dict['loss'].item():.4f} | Gen Len: {loss_dict['gen_len']}")
                print(f"    Acceptance Rate: {rollout_stats.get('avg_acceptance_rate', 0):.1%}")
                print(f"    Teacher Forward Calls: {rollout_stats.get('teacher_forward_calls', 0)}")
                print(f"    Entropy Gap (S-T): {loss_dict['student_entropy'] - loss_dict['teacher_entropy']:.3f}")
                print(f"    Time: {sample_time:.2f}s")
                print(f"    Generated (first 100 chars): {gen_str[:100]}...")
            
            # Gradient accumulation step
            if (sample_idx + 1) % config.gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(student_model.parameters(), config.max_grad_norm)
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                
                global_step += 1
                avg_loss = accumulated_loss / accumulated_samples
                avg_gen_len = sum(step_gen_lens) / len(step_gen_lens)
                avg_teacher_ent = sum(step_teacher_entropies) / len(step_teacher_entropies)
                avg_student_ent = sum(step_student_entropies) / len(step_student_entropies)
                avg_acceptance_rate = sum(step_acceptance_rates) / len(step_acceptance_rates)
                avg_teacher_calls = sum(step_teacher_forward_calls) / len(step_teacher_forward_calls)
                
                # Print step summary
                print(f"\n  >>> [Step {global_step}] Optimizer Step <<<")
                print(f"      Avg Loss (accum): {avg_loss:.4f}")
                print(f"      Avg Gen Len: {avg_gen_len:.1f}")
                print(f"      Avg Acceptance Rate: {avg_acceptance_rate:.1%}")
                print(f"      Avg Teacher Forward Calls: {avg_teacher_calls:.1f}")
                print(f"      Avg Entropy Gap: {avg_student_ent - avg_teacher_ent:.3f}")
                print(f"      Learning Rate: {scheduler.get_last_lr()[0]:.2e}")
                
                # Log to training_log
                log_dict = {
                    "train/loss": avg_loss,
                    "train/learning_rate": scheduler.get_last_lr()[0],
                    "train/epoch": epoch,
                    "train/global_step": global_step,
                    "train/gen_len_avg": avg_gen_len,
                    "train/teacher_entropy": avg_teacher_ent,
                    "train/student_entropy": avg_student_ent,
                    "train/entropy_gap": avg_student_ent - avg_teacher_ent,
                    "train/acceptance_rate": avg_acceptance_rate,
                    "train/teacher_forward_calls": avg_teacher_calls,
                }
                training_log.append({'step': global_step, 'epoch': epoch, **log_dict})
                
                # Reset accumulators
                accumulated_loss = 0.0
                accumulated_samples = 0
                step_gen_lens = []
                step_teacher_entropies = []
                step_student_entropies = []
                step_acceptance_rates = []
                step_teacher_forward_calls = []
        
        # ========== END OF EPOCH ==========
        epoch_time = time.time() - epoch_start_time
        epoch_avg_loss = sum(epoch_losses) / len(epoch_losses) if epoch_losses else 0
        epoch_avg_gen_len = sum(epoch_gen_lens) / len(epoch_gen_lens) if epoch_gen_lens else 0
        epoch_avg_acceptance = sum(epoch_acceptance_rates) / len(epoch_acceptance_rates) if epoch_acceptance_rates else 0
        
        print(f"\n{'='*70}")
        print(f"[EPOCH {epoch} SUMMARY]")
        print(f"{'='*70}")
        print(f"  Time: {epoch_time/60:.2f} min ({epoch_time:.0f} sec)")
        print(f"  Samples processed: {len(train_data)}")
        print(f"  Avg Loss: {epoch_avg_loss:.4f}")
        print(f"  Avg Gen Len: {epoch_avg_gen_len:.1f}")
        print(f"  Avg Acceptance Rate: {epoch_avg_acceptance:.1%}")
        print(f"  Global Step: {global_step}")
        
        # ========== EPOCH VALIDATION ==========
        print(f"\n{'-'*70}")
        print(f"[EPOCH {epoch} VALIDATION]")
        print(f"{'-'*70}")
        
        val_results = validate_with_details(
            teacher_model, student_model, val_data,
            config, tokenizer, num_generate_samples=3
        )
        
        print(f"\n  Val Loss: {val_results['val_loss']:.4f}")
        print(f"  Val Gen Len Avg: {val_results['val_gen_len_avg']:.1f}")
        print(f"  Val Acceptance Rate: {val_results.get('val_acceptance_rate', 0):.1%}")
        print(f"  Val Teacher Entropy: {val_results['val_teacher_entropy']:.3f}")
        print(f"  Val Student Entropy: {val_results['val_student_entropy']:.3f}")
        
        # Show sample generations
        print(f"\n  [Sample Generations from Validation]")
        for i, s in enumerate(val_results['samples']):
            print(f"\n  --- Val Sample {i+1} ({s['category']}) ---")
            print(f"  Query: {s['query'][:80]}...")
            print(f"  Gen Len: {s['gen_len']} tokens, AccRate: {s.get('acceptance_rate', 0):.1%}")
            print(f"  Generated:\n    {s['generated'][:300]}...")
        
        # Save best model
        if val_results['val_loss'] < best_val_loss:
            best_val_loss = val_results['val_loss']
            print(f"\n  *** NEW BEST VAL LOSS: {best_val_loss:.4f} ***")
            save_checkpoint(student_model, tokenizer, config, epoch, global_step, best_val_loss)
        
        # Save checkpoint per epoch
        if config.save_every_epoch:
            save_checkpoint(student_model, tokenizer, config, epoch, global_step, epoch_avg_loss)
        
        print(f"\n  Best Val Loss so far: {best_val_loss:.4f}")
        print(f"{'='*70}\n")
        
        student_model.train()
    
    return best_val_loss, training_log

print("[Training function defined with OPTIMIZED rollout]")

[Training function defined with OPTIMIZED rollout]


In [13]:
# Cell 12: Execute Training

print("\n" + "="*70)
print("   LUPI-SKD TRAINING SESSION")
print("="*70)

print("\n[DATA]")
print(f"  Train samples: {len(train_data)}")
print(f"  Val samples: {len(val_data)}")

print("\n[MODEL]")
print(f"  Teacher: {config.teacher_model_name}")
print(f"  Student: {config.student_model_name}")

print("\n[SKD HYPERPARAMETERS]")
print(f"  Top-K: {config.top_k}")
print(f"  Gamma (block size): {config.gamma}")
print(f"  Teacher Temperature: {config.teacher_temperature}")
print(f"  Student Temperature: {config.student_temperature}")

print("\n[TRAINING]")
print(f"  Epochs: {config.epochs}")
print(f"  Learning Rate: {config.learning_rate}")
print(f"  Gradient Accumulation: {config.gradient_accumulation_steps}")
print(f"  Max New Tokens: {config.max_new_tokens}")
print(f"  Steps per Epoch: {steps_per_epoch}")
print(f"  Total Steps: {total_steps}")
print(f"  Warmup Steps: {int(total_steps * config.warmup_ratio)}")

print("\n[CHECKPOINT]")
print(f"  Save directory: {config.checkpoint_dir}")
print(f"  Save every epoch: {config.save_every_epoch}")

print("\n" + "="*70)
print("   STARTING TRAINING...")
print("="*70 + "\n")

training_start_time = time.time()

best_val_loss, training_log = train_lupi_skd(
    teacher_model=teacher_model,
    student_model=student_model,
    train_data=train_data,
    val_data=val_data,
    config=config,
    tokenizer=tokenizer,
    optimizer=optimizer,
    scheduler=scheduler,
)

total_training_time = time.time() - training_start_time

print("\n" + "="*70)
print("   TRAINING COMPLETED!")
print("="*70)
print(f"\n  Total Time: {total_training_time/60:.2f} min ({total_training_time/3600:.2f} hours)")
print(f"  Best Val Loss: {best_val_loss:.4f}")
print(f"  Total Steps: {len(training_log)}")

# Summary of training progression
if training_log:
    first_loss = training_log[0]['train/loss']
    last_loss = training_log[-1]['train/loss']
    print(f"\n  Loss Progression:")
    print(f"    First step: {first_loss:.4f}")
    print(f"    Last step:  {last_loss:.4f}")
    print(f"    Change:     {last_loss - first_loss:.4f} ({(last_loss/first_loss - 1)*100:+.1f}%)")

print("="*70 + "\n")


   LUPI-SKD TRAINING SESSION

[DATA]
  Train samples: 180
  Val samples: 20

[MODEL]
  Teacher: Qwen/Qwen3-32B
  Student: Qwen/Qwen3-4B-Thinking-2507

[SKD HYPERPARAMETERS]
  Top-K: 5
  Gamma (block size): 30
  Teacher Temperature: 0.2
  Student Temperature: 0.5

[TRAINING]
  Epochs: 3
  Learning Rate: 2e-06
  Gradient Accumulation: 4
  Max New Tokens: 8192
  Steps per Epoch: 45
  Total Steps: 135
  Warmup Steps: 13

[CHECKPOINT]
  Save directory: /home/eoeldroal/WorkPlace/SKD_RAG_K/checkpoints/lupi_skd
  Save every epoch: True

   STARTING TRAINING...


[INITIAL VALIDATION] Before training starts
  Running validation on 20 samples (using optimized rollout)...


Validation:   0%|          | 0/20 [00:00<?, ?it/s]

KeyboardInterrupt: 

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x79d5189b0190>> (for post_run_cell), with arguments args (<ExecutionResult object at 79d51be62a10, execution_count=13 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 79d51be62510, raw_cell="# Cell 12: Execute Training

print("\n" + "="*70)
.." transformed_cell="# Cell 12: Execute Training

print("\n" + "="*70)
.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2B7b22686f73744e616d65223a2249534453227d/home/eoeldroal/WorkPlace/SKD_RAG_K/notebooks/train_lupi_skd_full.ipynb#X55sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


ConnectionResetError: Connection lost

In [ ]:
# Cell 13: Final Evaluation and Sample Generation

def final_evaluation(student_model, val_data, config, tokenizer):
    """Generate outputs for all validation samples."""
    student_model.eval()
    device = next(student_model.parameters()).device
    results = []
    
    print("Generating final evaluation samples...")
    for sample in tqdm(val_data, desc="Final Evaluation"):
        student_inputs = tokenizer.apply_chat_template(
            [{"role": "user", "content": sample['query']}],
            return_dict=True,
            tokenize=True,
            return_tensors="pt",
            add_generation_prompt=True,
            enable_thinking=True,
        )
        
        with torch.no_grad():
            generated = student_model.generate(
                input_ids=student_inputs['input_ids'].to(device),
                max_new_tokens=config.max_new_tokens,
                temperature=0.7,
                top_p=0.9,
            )
        
        gen_text = tokenizer.decode(
            generated[0][student_inputs['input_ids'].shape[1]:], 
            skip_special_tokens=True
        )
        
        results.append({
            'id': sample.get('id', 'unknown'),
            'category': sample.get('category', 'unknown'),
            'query': sample['query'],
            'knowledge': sample['knowledge'],
            'generated': gen_text,
        })
    
    # Save to file
    output_path = Path(config.checkpoint_dir) / "final_val_generations.json"
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    
    # Save training log
    log_path = Path(config.checkpoint_dir) / "training_log.json"
    with open(log_path, 'w', encoding='utf-8') as f:
        json.dump(training_log, f, indent=2)
    
    print(f"\n[Final Evaluation Complete]")
    print(f"  Saved {len(results)} generations to {output_path}")
    print(f"  Saved training log to {log_path}")
    
    return results

final_results = final_evaluation(student_model, val_data, config, tokenizer)

# Show sample results
print("\n[Sample Generation Results]")
for i, result in enumerate(final_results[:3]):
    print(f"\n--- Sample {i+1} ({result['category']}) ---")
    print(f"Query: {result['query'][:100]}...")
    print(f"Generated: {result['generated'][:300]}...")

# Finish WandB
wandb.finish()
print("\n[WandB run finished]")